Preprocessing, EDA and FE all are completed now  
Next, we will train our model with lightgbm  
We are not normalizing our data, i.e not changing the distribution and tree methods work very well with these un-normalized values  
Plus LGBM is very fast and is a serial kaggle competition winner

In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import pickle
from sklearn import model_selection

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
X_test = pd.read_pickle('np/df_test.pickle')
X_train = pd.read_pickle('np/df.pickle')

y_train = X_train['exited'].values
y_test = X_test['exited'].values

In [3]:
X_train = X_train.drop('exited', axis = 1)
X_test = X_test.drop('exited', axis = 1)

In [4]:
def run_lgb(train_X, train_y, val_X, val_y, test_X):
    params = {
        "objective" : "binary",
        "metric" : "binary_logloss",
        "num_leaves" : 20,
        "learning_rate" : 0.007,
        "bagging_fraction" : 0.6,
        "feature_fraction" : 0.6,
        "bagging_freq" : 5,
        "bagging_seed" : 2018,
        "verbosity" : -1
    }
    
    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(params, lgtrain, 1000, valid_sets=[lgtrain, lgval], early_stopping_rounds=100, 
                      verbose_eval=200, evals_result=evals_result)
    
    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    return pred_test_y, model, evals_result

In [5]:
# Training LGB
seeds = [42, 2018]
pred_test_full_seed = 0
for seed in seeds:
    kf = model_selection.KFold(n_splits=5, shuffle=True, random_state=seed)
    pred_test_full = 0
    for dev_index, val_index in kf.split(X_train):
        dev_X, val_X = X_train.loc[dev_index,:], X_train.loc[val_index,:]
        dev_y, val_y = y_train[dev_index], y_train[val_index]
        pred_test, model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y, X_test)
        pred_test_full += pred_test
    pred_test_full /= 5.
    pred_test_full = np.expm1(pred_test_full)
    pred_test_full_seed += pred_test_full
    print("Seed {} completed....".format(seed))
pred_test_full_seed /= np.float(len(seeds))

print("LightGBM Training Completed...")

Training until validation scores don't improve for 100 rounds.
[200]	training's binary_logloss: 0.376959	valid_1's binary_logloss: 0.375503
[400]	training's binary_logloss: 0.283317	valid_1's binary_logloss: 0.281733
[600]	training's binary_logloss: 0.243527	valid_1's binary_logloss: 0.243311
[800]	training's binary_logloss: 0.222816	valid_1's binary_logloss: 0.22637
[1000]	training's binary_logloss: 0.209904	valid_1's binary_logloss: 0.217825
Did not meet early stopping. Best iteration is:
[1000]	training's binary_logloss: 0.209904	valid_1's binary_logloss: 0.217825
Training until validation scores don't improve for 100 rounds.
[200]	training's binary_logloss: 0.375016	valid_1's binary_logloss: 0.383526
[400]	training's binary_logloss: 0.280354	valid_1's binary_logloss: 0.29425
[600]	training's binary_logloss: 0.239134	valid_1's binary_logloss: 0.258185
[800]	training's binary_logloss: 0.217876	valid_1's binary_logloss: 0.242845
[1000]	training's binary_logloss: 0.204616	valid_1's bin

In [6]:
# feature importance
print("Features Importance...")
gain = model.feature_importance('gain')
featureimp = pd.DataFrame({'feature':model.feature_name(), 
                   'split':model.feature_importance('split'), 
                   'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(featureimp)

Features Importance...
         feature  split       gain
12        active   1282  22.740450
8            age   2679  18.445806
4             c3   1385   8.746731
9        balance   2982   6.974028
14           p_2    652   6.680613
0        spanish   1213   6.216665
11      has_card   1269   5.749348
10      products   1183   5.136999
1         german    987   4.648748
7         gender    711   4.574573
5             c4   1055   4.018991
6   credit_score   2035   2.005409
2             c1    674   1.644750
13           p_1    281   1.217403
3             c2    482   0.743930
15           p_3    130   0.455555


In [7]:
pred = model.predict(X_test.values)

In [8]:
pred

array([0.04974974, 0.04771815, 0.17825329, ..., 0.8091257 , 0.15507791,
       0.40616478])

In [9]:
pred[pred < 0.5] = 0
pred[pred >= 0.5] = 1

In [12]:
sum(y_test)

393

y_test has less than 20% exited cases. So, kappa metric should be used instead of accuracy

In [10]:
from sklearn.metrics import cohen_kappa_score

In [11]:
cohen_kappa_score(y_test, pred)

0.52827210808005

Kappa can be in the range -1 and +1  
Anything above +0.5 is a strong agreement